# Визуализация алгоритма DES

In [224]:
import json
from functools import reduce


def get_bit(number, bit):
    return 1 & (number >> bit)

def rearrange_bits_by_matrix(number, matrix):
    return reduce(lambda a, i: (a << 1) | get_bit(number, i - 1), matrix, 0)

def split(number, mid_bit, parts):
    return reduce(lambda a, i: a + [(number >> (i * mid_bit)) & ((1 << mid_bit) - 1)], reversed(range(0, parts)), [])

def merge(numbers, mid_bit):
    return reduce(lambda a, i: (a << mid_bit) | (i & ((1 << mid_bit) - 1)), numbers, 0)

def cyclic_shift(number, bits, limit):
    return ((number << bits) | ((number << bits) >> limit)) & ((1 << limit) - 1)

In [225]:
with open("DES_tables.json") as DES_tables:
    tables = json.load(DES_tables)

In [226]:
def prepare_data_from_string(string, length):
    print(f"Preparing string: '{string}'")
    bts = bytearray(string, "ascii")[:length // 8]
    print(" ".join(string), " ".join(str(i) for i in bts), " ".join(format(i, '08b') for i in bts), sep='\n')
    output = reduce(lambda a, i: (a << 8) + i, bts, 0)
    print(format(output, f'0{length}b'), output, "", sep='\n')
    return output

print("Input text: first and last names, 64bit")
plain_text = prepare_data_from_string("Name_Surname", 64)
print("Key: uni number and fathers name first letter, 56bit")
key = prepare_data_from_string("830433L", 56)

Input text: first and last names, 64bit
Preparing string: 'Name_Surname'
N a m e _ S u r n a m e
78 97 109 101 95 83 117 114
01001110 01100001 01101101 01100101 01011111 01010011 01110101 01110010
0100111001100001011011010110010101011111010100110111010101110010
5647915689857742194

Key: uni number and fathers name first letter, 56bit
Preparing string: '830433L'
8 3 0 4 3 3 L
56 51 48 52 51 51 76
00111000 00110011 00110000 00110100 00110011 00110011 01001100
00111000001100110011000000110100001100110011001101001100
15818880823014220



### Шаг 1: начальная перестановка данных

In [227]:
print("Input text initial permutation:\n", format(plain_text, '064b'), sep='\n')
prepared_text = rearrange_bits_by_matrix(plain_text, tables['ip'])
print("\nis changed to\n", format(prepared_text, '064b'), sep='\n')

Input text initial permutation:

0100111001100001011011010110010101011111010100110111010101110010

is changed to

1000110110101000011100110000000001111110101110100000111111111111


### Шаг 2: начальная перестановка ключа

In [228]:
print("Key permuted choice:\n", format(key, '056b'), sep='\n')
prepared_key = rearrange_bits_by_matrix(key, tables["pc1"])
print("\nis changed to\n", format(prepared_key, '056b'), sep='\n')

Key permuted choice:

00111000001100110011000000110100001100110011001101001100

is changed to

00100110001001100000100101000000000101111110011111100001


### Шаг 2: разделение и циклический сдвиг ключа

In [229]:
print("Key separation:\n", format(prepared_key, '056b'), sep='\n')
key_l, key_r = split(prepared_key, 28, 2)
print("\nis split to\n", f"{format(key_l, '028b')} {format(key_r, '028b')}", "\n", sep='\n')

shifted_keys = {}
print("Key shift (there are only two shift options, by 1 bit and by 2 bits):\n")
for i in set(tables["sls"]):
    print(f"{format(key_l, '028b')} {format(key_r, '028b')}", f"\nis shifted by {i} to\n", sep='\n')
    sh_key_l = cyclic_shift(key_l, i, 28)
    sh_key_r = cyclic_shift(key_r, i, 28)
    print(f"{format(sh_key_l, '028b')} {format(sh_key_r, '028b')}", "", sep='\n')
    shifted_keys[i] = (sh_key_l, sh_key_r)

Key separation:

00100110001001100000100101000000000101111110011111100001

is split to

0010011000100110000010010100 0000000101111110011111100001


Key shift (there are only two shift options, by 1 bit and by 2 bits):

0010011000100110000010010100 0000000101111110011111100001

is shifted by 1 to

0100110001001100000100101000 0000001011111100111111000010

0010011000100110000010010100 0000000101111110011111100001

is shifted by 2 to

1001100010011000001001010000 0000010111111001111110000100



### Шаг 3: генерация раундовых ключей

In [230]:
print("Key generation:\n")
keys = [None] * 16

for i in range(0, 16):
    print(f"Generating key for round {i}...")
    pre_key = merge([shifted_keys[tables["sls"][i]][0], shifted_keys[tables["sls"][i]][1]], 28)
    print("\nPermuted choice:\n", format(pre_key, '056b'), sep='\n')
    keys[i] = rearrange_bits_by_matrix(pre_key, tables["pc2"])
    print("\nis changed to\n", format(keys[i], '048b'), "\n", sep='\n')

Key generation:

Generating key for round 0...

Permuted choice:

01001100010011000001001010000000001011111100111111000010

is changed to

011000001001011001110101010111001000100010000001


Generating key for round 1...

Permuted choice:

01001100010011000001001010000000001011111100111111000010

is changed to

011000001001011001110101010111001000100010000001


Generating key for round 2...

Permuted choice:

10011000100110000010010100000000010111111001111110000100

is changed to

011000100011111001100110010000000111100101000000


Generating key for round 3...

Permuted choice:

10011000100110000010010100000000010111111001111110000100

is changed to

011000100011111001100110010000000111100101000000


Generating key for round 4...

Permuted choice:

10011000100110000010010100000000010111111001111110000100

is changed to

011000100011111001100110010000000111100101000000


Generating key for round 5...

Permuted choice:

10011000100110000010010100000000010111111001111110000100

is changed 

### Шаг 4: Функция f

In [231]:
def f(value, key):
    print("Expanding and XORing with key R part:\n", format(value, '032b'), sep='\n')
    expanded_value = rearrange_bits_by_matrix(value, tables["e"])
    print("\nexpanded to\n", format(expanded_value, '048b'), sep='\n')
    xor = expanded_value ^ key
    b_parts = split(xor, 6, 8)
    print("\nXORed to\n", format(xor, '048b'), " ".join(format(i, '06b') for i in b_parts), "", sep='\n')
    nums = [None] * 8
    print("Applying S boxes:\n", " ".join(format(i, '06b') for i in b_parts), sep='\n')
    for index, part in enumerate(b_parts):
        row = (get_bit(part, 5) << 1) | (get_bit(part, 0))
        col = reduce(lambda a, b: (a << 1) | b, [get_bit(part, i) for i in reversed(range(1, 5))], 0)
        nums[index] = tables["s"][index][row * 16 + col]
    print("\nchanged to\n", " ".join(format(i, '04b') for i in nums), sep='\n')
    pre_result = merge(nums, 4)
    print("\nconcatenated to\n", format(pre_result, '032b'), "", sep='\n')
    print("R permutation:\n", format(pre_result, '032b'), sep='\n')
    result = rearrange_bits_by_matrix(pre_result, tables["p"])
    print("\nchanged to\n", format(result, '032b'), sep='\n')
    return result

### Шаг 5: раунды шифрования

In [232]:
print("DES main body:\n")
L, R = split(prepared_text, 32, 2)

for i in range(0, 16):
    print(f"Round {i}...\n", f"L: {format(L, '032b')}, R: {format(R, '032b')}", "", sep='\n')
    old_R = R
    R = f(R, keys[i]) ^ L
    L = old_R
    print("", f"L: {format(L, '032b')}, R: {format(R, '032b')}", "\n", sep='\n')

DES main body:

Round 0...

L: 10001101101010000111001100000000, R: 01111110101110100000111111111111

Expanding and XORing with key R part:

01111110101110100000111111111111

expanded to

011111111111111110100000001011111010101111111101

XORed to

000111110110100111010101011100110010001101111100
000111 110110 100111 010101 011100 110010 001101 111100

Applying S boxes:

000111 110110 100111 010101 011100 110010 001101 111100

changed to

0100 0110 0000 0010 1110 0000 0001 0101

concatenated to

01000110000000101110000000010101

R permutation:

01000110000000101110000000010101

changed to

10000000110111100001011000000000

L: 01111110101110100000111111111111, R: 00001101011101100110010100000000


Round 1...

L: 01111110101110100000111111111111, R: 00001101011101100110010100000000

Expanding and XORing with key R part:

00001101011101100110010100000000

expanded to

000000000001010100001100001101011101010110100000

XORed to

011000001000001101111001011010010101110100100001
011000 001000 

In [233]:
print("Final swap:\n")
L, R = R, L
print(f"L: {format(L, '032b')}, R: {format(R, '032b')}")
output_text = merge([L, R], 32)
print("", format(output_text, '064b'), sep='\n')

Final swap:

L: 00101101100000000000000001011011, R: 11001100101000110000110010101010

0010110110000000000000000101101111001100101000110000110010101010


### Шаг 6: финальная перестановка данных

In [234]:
print("Input text final permutation:\n", format(output_text, '064b'), sep='\n')
cipher_text = rearrange_bits_by_matrix(output_text, tables['ip-1'])
print("\nis changed to\n", format(cipher_text, '064b'), sep='\n')

Input text final permutation:

0010110110000000000000000101101111001100101000110000110010101010

is changed to

0100110110000001010001101000000011010011000100111100010010000110
